In [1]:
# if lights are off when a color transition starts, or for a brightness transition to zero, transition will fail.
# So we need a way to initiate a transition based on expected state.

# But reworking the script to do that is... kinda big. This is the quick and dirty version.
# Assumes the transition is 1hr+ long. 

In [17]:
import datetime
import random
import time

from tradfri import Tradfri

Todo:
look for all 'todo' instances here.


figure out multithreading. https://stackoverflow.com/questions/2846653/how-to-use-threading-in-python

#### Approach:

Process:
Plan transition for [brightness, color] for each light based on start & end times
Skip through transition until we reach current time. Do short transition (30s?) to that. 


In [12]:
### DEBUG
from pprint import pprint

In [13]:
### CONFIG:

evening_start_time = datetime.time(hour=17, minute=30)

# 'quick' initial transition to bring lights in line with overall plan. think a minute-ish.
# definitely don't go more than 15 min or you're not 'catching up' fast enough.
initial_transition_duration = datetime.timedelta(seconds=60)

# different times for end of color transition and end of brightness transition:
evening_color_end_value = 2200
evening_color_end_time = datetime.time(hour=23)

evening_brightness_end_value = 0
evening_brightness_end_time = datetime.time(hour=23, minute=45)

# NOTE: don't transition over midnight, or this will break. Like I said, quick and dirty.
#(Modify to use datetimes if you need to transition spanning midnight.
#  Or check if start is after end & +1 the date.)


# we'll extend transitions by a random amount, between 0 and this many seconds,
#  to prevent everything from switching off at once.
# recommend 5 minutes = 300 seconds
random_delay_range = 300



In [4]:
# basically just a record of daytime values for the office. also defined in homeassistant as an automation.
# *not* the same names as in light-schedule.py; these are more like the entity ids.
daytime_values = {"office_table": {
                            "brightness": 230,
                            "color": 4000
                            },
           "desk_lamp": {
                           "brightness": 200,
                           "color": 3200
                       },
           "geo_desk": {
                           "brightness": 130,
                           "color": 3350
                       },
           "floor_uplight": {
                           "brightness": 230,
                           "color": 3800
                       },
           "monitor_left": {
                           "brightness": 130,
                           "color": 3700
                       },
           "monitor_right": {
                           "brightness": 130,
                           "color": 3700
                       }
          }


In [5]:
# Do some setup / initial math and stuff.

today = datetime.datetime.now().date()

# make datetimes from dates. Use these internally instead of vals above.
e_start = datetime.datetime.combine(today, evening_start_time)
c_end = datetime.datetime.combine(today, evening_color_end_time)
b_end = datetime.datetime.combine(today, evening_brightness_end_time)

# timedeltas
color_duration = c_end - e_start
brightness_duration = b_end - e_start

# put these into a more easily programmatically-navigable structure.
details = {
              "color": {
                        "value": evening_color_end_value,
                        "end_time": evening_color_end_time,
                        "duration": color_duration
                       },
              "brightness": {
                        "value":evening_brightness_end_value,
                        "end_time": evening_brightness_end_time,
                        "duration": brightness_duration
              }
}


In [6]:
# list of each transition we'll have to do. these are *start* values.
transitions = []
for i in daytime_values:
    #print(i)
    transitions.append({"name": i, "type": "brightness", "value": daytime_values[i]['brightness']})
    transitions.append({"name": i, "type": "color", "value": daytime_values[i]['color']})

In [7]:
transitions[0]

{'name': 'office_table', 'type': 'brightness', 'value': 230}

In [ ]:
### multithreading demo here

In [95]:
def get_val(transition):
    entity = 'light.' + transition['name']
    # instantiate
    device = Tradfri(entity)
    
    if transition['type'] == 'brightness':
        val = device.get_brightness()
    else: # it's color
        val = device.get_temp_kelvin()

    #print("gv()", transition['name'], transition['type'], val, sep='\t')
    return (transition['name'], transition['type'], val)

In [89]:
# non-multithreaded version
for i in transitions[0:3]:
    val = get_val(i)
    #print(i['name'], i['type'], val[1])

gv	office_table	brightness	154
gv	office_table	color	2786
gv	desk_lamp	brightness	135


In [41]:
from multiprocessing.dummy import Pool as ThreadPool
# maybe problematic if we have hundreds of transitions, but we don't
pool = ThreadPool(len(transitions))

In [93]:
# multithreaded version. this is actually pretty easy.
results = pool.map(get_val, transitions)

gv	floor_uplight	brightness	152
gv	desk_lamp	color	2538
gv	geo_desk	color	2571
gv	floor_uplight	color	2747
gv	geo_desk	brightness	87
gv	monitor_left	brightness	87
gv	monitor_left	color	2725
gv	office_table	color	2762
gv	desk_lamp	brightness	134
gv	monitor_right	brightness	88
gv	office_table	brightness	153
gv	monitor_right	color	2740


In [94]:
results

[('office_table', 'brightness', 153),
 ('office_table', 'color', 2762),
 ('desk_lamp', 'brightness', 134),
 ('desk_lamp', 'color', 2538),
 ('geo_desk', 'brightness', 87),
 ('geo_desk', 'color', 2571),
 ('floor_uplight', 'brightness', 152),
 ('floor_uplight', 'color', 2747),
 ('monitor_left', 'brightness', 87),
 ('monitor_left', 'color', 2725),
 ('monitor_right', 'brightness', 88),
 ('monitor_right', 'color', 2740)]

In [ ]:
## end multithreading demo

In [16]:
# ok, here we go

def catch_up(transition):
    # 'resumes' an existing transition.
    
    # step 1 in randomizing
    time.sleep(random.uniform(0,9))
    
    
    ### todo: check if current time is between start and end times before going through the whole plan.
    

    #print(i)

    entity = 'light.' + transition['name']
    # instantiate
    device = Tradfri(entity)
    
    #print(details[i['type']])
    transition_type = transition['type']
    final_attr = {transition_type: details[transition_type]['value']}
    duration = details[transition_type]['duration']
    
    #print("final:", final_attr)
    
    # usage:
    # plan_transition(self, new_attr, duration, start_time=None)
    
    # randomize a bit more.
    duration + random.uniform(0, random_delay_range)
    plan = device.plan_transition(final_attr, duration, e_start)
    
    # debug
    pprint(plan['plan'][-3:])
    
    # find the point in the the transition *after* our initial transition to current value
    benchmark_time = datetime.datetime.now() + initial_transition_duration
    
    current_step_no = None
    # find out how far into the transition we are.
    for i in plan['plan']: 
        if i['step_start_time'] >= benchmark_time:
            current_step_no = i['step_number'] - 1
            break
    
    if current_step_no is None:
        # probably beyond the end of the transition.
        print("no step found!")
        
        ### write me
        
    elif current_step_no == 0:
        # may or may not have started yet? do we need to special case this?
        print("cur step is zero!")
        
        ### write me
        
    elif current_step_no < 0:
        # definitely haven't started yet
        print("cur step is negative! step:", current_step_no)
        
        ### write me
    
    else: #continue
    
        current_step_vals = plan['plan'][current_step_no]

        print("cur step:", current_step_vals)

        cur_attr = {transition_type: plan['plan'][current_step_no][transition_type]}

        print("ca:", cur_attr)

        # Do a quick transition to these values
        device.transition(cur_attr, initial_transition_duration)

        # Then, resume normal transition, starting from current step / values.
        device.execute_transition(plan[current_step_vals:])
    

[{'brightness': 1.0,
  'step_number': 0,
  'step_start_time': datetime.datetime(2018, 9, 9, 17, 30)},
 {'brightness': 0.0,
  'step_number': 1,
  'step_start_time': datetime.datetime(2018, 9, 9, 20, 37, 30)}]
no step found!


In [ ]:
for i in transitions[0:1]:
    catch_up(i)

In [29]:

time.sleep(random.uniform(0,11))
print(ok)

NameError: name 'ok' is not defined